In [1]:
import os
import subprocess
import time
from pathlib import Path
import geopandas as gpd
gpd._compat.USE_PYGEOS = False
import snail.intersection
import snail.io
import snkit
import snkit.network
from pyproj import Geod
import rasterio
import numpy as np

In [3]:
# Set paths
road_path = r"D:\projects\sovereign-risk\Thailand\data\infrastructure\OSM\thailand-latest-free.shp\gis_osm_roads_free_1.shp"

In [4]:
# Read file
roads = gpd.read_file(road_path)

In [8]:
# Clean the data
# keep only the specified columns
roads = roads[["osm_id", "fclass", "name", "geometry"]]
# keep only the roads whose "fclass" is inthe list
roads = roads[
    roads.fclass.isin(
    ["motorway",
    "motorway_link",
    "trunk",
    "trunk_link",
    "primary",
    "primary_link",
    "secondary",
    "secondary_link",
    "tertiary",
    "tertiary_link",
    ]
    )
]
# Rename some columns
roads = roads.rename(
    columns={
    "fclass": "road_type",
    }
)

In [16]:
roads.to_file(r"D:\projects\sovereign-risk\Thailand\data\infrastructure\gridded\analysis\roads\cleaned_roads.json", driver="GeoJSON")

In [2]:
# Load split roads
split_road_path = r"D:\projects\sovereign-risk\Thailand\data\infrastructure\gridded\analysis\roads\split_roads.geojson"
split_roads = gpd.read_file(split_road_path)

In [3]:
# Calculate length
geod = Geod(ellps="WGS84")
split_roads['length_m'] = split_roads.geometry.apply(geod.geometry_length)

In [4]:
split_roads

,osm_id,road_type,name,split,i_0,j_0,index_i,index_j,geometry,length_m
0,8654614,secondary,ถนนพิษณุโลก,0,3803,8041,3803,8041,"LINESTRING (100.51344 13.76333, 100.51353 13.7...",95.141903
1,8654614,secondary,ถนนพิษณุโลก,1,3804,8041,3804,8041,"LINESTRING (100.51417 13.76287, 100.51421 13.7...",62.911296
2,8654614,secondary,ถนนพิษณุโลก,2,3804,8042,3804,8042,"LINESTRING (100.51461 13.76250, 100.51500 13.7...",55.176777
3,8654614,secondary,ถนนพิษณุโลก,3,3805,8042,3805,8042,"LINESTRING (100.51500 13.76218, 100.51540 13.7...",77.900346
4,8668134,motorway,ทางพิเศษบูรพาวิถี,0,4321,8279,4321,8279,"LINESTRING (100.94520 13.56494, 100.94500 13.5...",22.342456
...,...,...,...,...,...,...,...,...,...,...
2181691,1282730569,tertiary,None,1,4724,16705,4724,16705,"LINESTRING (101.28128 6.54250, 101.28162 6.54303)",70.020426
2181692,1282730571,primary,ถนนเทศบาล 1,0,4721,16704,4721,16704,"LINESTRING (101.27851 6.54379, 101.27885 6.54333)",63.011905
2181693,1282730571,primary,ถนนเทศบาล 1,1,4721,16705,4721,16705,"LINESTRING (101.27885 6.54333, 101.27907 6.543...",58.703741
2181694,1282730571,primary,ถนนเทศบาล 1,2,4722,16705,4722,16705,"LINESTRING (101.27917 6.54291, 101.27947 6.54250)",55.998532


In [18]:
# Load raster geometry that we want to aggregate vector to
flood_map = r"D:\projects\sovereign-risk\Thailand\data\flood\maps\THA_global_pc_h100glob.tif"
with rasterio.open(flood_map) as dataset:
    width = dataset.width
    height = dataset.height
    transform = dataset.transform
    crs = dataset.crs

In [7]:
print('width', width)
print('height', height)
print('transform', transform)
print('crs', crs)

width 9953
height 17817
transform | 0.00, 0.00, 97.34|
| 0.00,-0.00, 20.46|
| 0.00, 0.00, 1.00|
crs EPSG:4326


In [8]:
# Code for burning road length dataframe onto raster (from Tom)

In [12]:
# Calculate cell index
splits_df = split_roads.copy()
length_per_cell = splits_df[['index_i', 'index_j', 'length_m']].groupby(['index_i', 'index_j']).sum().reset_index()
# Filter out any negative indices
length_per_cell = length_per_cell[(length_per_cell['index_i'] >= 0) & (length_per_cell['index_j'] >= 0)]

In [13]:
length_per_cell

,index_i,index_j,length_m
1,473,2989,11.892879
2,473,2990,50.441678
3,473,2991,90.502083
4,473,2992,53.497538
5,473,2993,110.942952
...,...,...,...
1768624,9946,5788,110.972582
1768625,9946,5789,32.638553
1768626,9947,5785,20.104857
1768627,9947,5786,92.540987


In [14]:
length_raster = np.zeros((height, width))

In [15]:
# Populate the raster with aggregated values
for row in length_per_cell.reset_index().itertuples():
    col = row.index_i
    row_idx = row.index_j
    length_raster[row_idx, col] = row.length_m

In [20]:
# Write raster to file
output_file = r"D:\projects\sovereign-risk\Thailand\data\infrastructure\gridded\final\road_lengths_m.tif"
with rasterio.open(
    output_file,
    'w',
    driver='GTiff',
    height = height,
    width = width,
    count = 1,
    dtype = np.float32,
    crs = crs,
    transform = transform,
    compress = 'lzw'
) as dataset:
    dataset.write(length_raster, 1)

In [21]:
# Do same for railways

In [22]:
# Set paths
rail_path = r"D:\projects\sovereign-risk\Thailand\data\infrastructure\OSM\thailand-latest-free.shp\gis_osm_railways_free_1.shp"

In [23]:
# Read file
rail = gpd.read_file(rail_path)

In [26]:
rail

,osm_id,code,fclass,name,layer,bridge,tunnel,geometry
0,9353290,6101,rail,None,0,F,F,"LINESTRING (100.52103 13.75727, 100.52110 13.7..."
1,23958080,6101,rail,สายใต้;สะพานพระรามหก,1,T,F,"LINESTRING (100.51820 13.81590, 100.51389 13.8..."
2,24007686,6101,rail,None,0,F,F,"LINESTRING (99.01716 18.77826, 99.01718 18.777..."
3,25031221,6101,rail,สะพานแควใหญ,1,T,F,"LINESTRING (99.50413 14.04170, 99.50313 14.03971)"
4,25031222,6101,rail,สายกาญจนบุรี,0,F,F,"LINESTRING (99.50313 14.03971, 99.50276 14.039..."
...,...,...,...,...,...,...,...,...
4852,1273148779,6101,rail,สายใต้,1,T,F,"LINESTRING (99.85391 11.99253, 99.85379 11.99225)"
4853,1273150600,6101,rail,สายใต้,0,F,F,"LINESTRING (99.91025 12.32337, 99.89645 12.30296)"
4854,1273150601,6101,rail,สายใต้,1,T,F,"LINESTRING (99.89645 12.30296, 99.89629 12.30274)"
4855,1273150602,6101,rail,สายใต้,0,F,F,"LINESTRING (99.91030 12.32333, 99.89651 12.30293)"


In [27]:
# Keep only the specified columns
rail = rail[['osm_id', 'fclass', 'name', 'geometry']]
# Keep only the railways whose "fclass" is in the list
rail = rail[
    rail.fclass.isin(
        [
        "rail",
        "light_rail",
        "tram",
        ]
    )
]
# Renam columns
rail = rail.rename(
    columns={
        "fclass": "rail_type",
    }
)

In [29]:
rail.to_file(r"D:\projects\sovereign-risk\Thailand\data\infrastructure\gridded\analysis\rail\cleaned_rail.json", driver="GeoJSON")

In [32]:
# Load split rail
split_rail_path = r"D:\projects\sovereign-risk\Thailand\data\infrastructure\gridded\analysis\rail\split_rail.geojson"
split_rail = gpd.read_file(split_rail_path)

In [33]:
split_rail

,osm_id,rail_type,name,split,i_0,j_0,index_i,index_j,geometry
0,9353290,rail,None,0,3812,8048,3812,8048,"LINESTRING (100.52103 13.75727, 100.52110 13.7..."
1,9353290,rail,None,1,3812,8047,3812,8047,"LINESTRING (100.52118 13.75750, 100.52127 13.7..."
2,9353290,rail,None,2,3813,8047,3813,8047,"LINESTRING (100.52167 13.75801, 100.52167 13.7..."
3,9353290,rail,None,3,3813,8046,3813,8046,"LINESTRING (100.52211 13.75833, 100.52224 13.7..."
4,9353290,rail,None,4,3814,8046,3814,8046,"LINESTRING (100.52250 13.75855, 100.52270 13.7..."
...,...,...,...,...,...,...,...,...,...
83243,1273150602,rail,สายใต้,38,3063,9791,3063,9791,"LINESTRING (99.89750 12.30438, 99.89735 12.30417)"
83244,1273150602,rail,สายใต้,39,3063,9792,3063,9792,"LINESTRING (99.89735 12.30417, 99.89679 12.30333)"
83245,1273150602,rail,สายใต้,40,3063,9793,3063,9793,"LINESTRING (99.89679 12.30333, 99.89667 12.30315)"
83246,1273150602,rail,สายใต้,41,3062,9793,3062,9793,"LINESTRING (99.89667 12.30315, 99.89651 12.30293)"


In [35]:
# Calculate length
geod = Geod(ellps="WGS84")
split_rail['length_m'] = split_rail.geometry.apply(geod.geometry_length)

In [36]:
# Load raster geometry that we want to aggregate vector to
flood_map = r"D:\projects\sovereign-risk\Thailand\data\flood\maps\THA_global_pc_h100glob.tif"
with rasterio.open(flood_map) as dataset:
    width = dataset.width
    height = dataset.height
    transform = dataset.transform
    crs = dataset.crs

In [37]:
# Calculate cell index
splits_df = split_rail.copy()
length_per_cell = splits_df[['index_i', 'index_j', 'length_m']].groupby(['index_i', 'index_j']).sum().reset_index()
# Filter out any negative indices
length_per_cell = length_per_cell[(length_per_cell['index_i'] >= 0) & (length_per_cell['index_j'] >= 0)]

In [38]:
length_raster = np.zeros((height, width))

In [39]:
# Populate the raster with aggregated values
for row in length_per_cell.reset_index().itertuples():
    col = row.index_i
    row_idx = row.index_j
    length_raster[row_idx, col] = row.length_m

In [40]:
# Write raster to file
output_file = r"D:\projects\sovereign-risk\Thailand\data\infrastructure\gridded\final\rail_lengths_m.tif"
with rasterio.open(
    output_file,
    'w',
    driver='GTiff',
    height = height,
    width = width,
    count = 1,
    dtype = np.float32,
    crs = crs,
    transform = transform,
    compress = 'lzw'
) as dataset:
    dataset.write(length_raster, 1)

In [30]:
# Test for other OSM maps
buildings_osm = r"D:\projects\sovereign-risk\Thailand\data\infrastructure\OSM\thailand-latest-free.shp\gis_osm_buildings_a_free_1.shp"
landuse_osm = r"D:\projects\sovereign-risk\Thailand\data\infrastructure\OSM\thailand-latest-free.shp\gis_osm_landuse_a_free_1.shp"

In [ ]:
buildings = gpd.read_file(buildings_osm)
landuse = gpd.read_file(landuse_osm)